# Transfer Learning

### Importing the libraries

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Input, Dense
from keras.models import load_model, Model
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
#tf.__version__

In [2]:
os.chdir ('Z:\PPMI_Data\Excels\CollaborativeFiltering')
df2 = pd.read_csv('NonMotor_Empty.csv')
df3 = df2.drop('Patient_ID', axis = 1)
new_age = int(input("What is the user's age (in years): "))
df3.at[0, 'Age'] = new_age
dframe1 = pd.read_csv('Feats45_unCategSparse_APPRDX.csv')
dframe = dframe1.drop(['PATNO','APPRDX','Age'], axis = 1)
dframe = dframe.add(1)
def update_values(df3, dframe):
    # Input from the user for number of columns
    num_columns = int(input("Enter the number of columns you want to update (minimum 3, maximum 45): "))
    
    while num_columns < 3 or num_columns > 45:
        print("Invalid number of columns. Please enter a number between 3 and 45.")
        num_columns = int(input("Enter the number of columns you want to update (minimum 3, maximum 45): "))
    
    columns = []
    values = []
     
    # Loop to get input from the user for each column
    for i in range(num_columns):
        column = input("Enter the name of the Feature %d: " % (i + 1))
        value = input("Enter the score of the Feature %d: " % (i + 1))
        
        columns.append(column)
        values.append(int(value)) 
        
    # Update the values in the dataframe
    for i in range(num_columns):
        df3.loc[df3[columns[i]] != values[i], columns[i]] = values[i] 
    
    # Put values equal to zero on the columns that are not selected
    for col in dframe.columns:
        if col not in columns and col != 'Age': # Skip 'Age' column
            dframe = dframe.drop(col, axis=1)
    
    return df3, dframe

df3, dframe = update_values(df3, dframe)
col_age = df3.pop('Age')
df3.insert(0, "Age", col_age)
df3 = df3.dropna(axis = 1)
dframe = dframe.dropna()
col_age = dframe1.pop('Age')
dframe.insert(0, "Age", col_age)
col_Apprdx = dframe1.pop('APPRDX')
dframe.insert(0, 'APPRDX', col_Apprdx)
dframes = dframe.drop(['APPRDX'], axis = 1)
dframe.loc[:, "APPRDX"] = dframe["APPRDX"].apply(lambda x: x - 1)
X = dframe.iloc[:, 1:].values
y = dframe.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

# Load the pre-trained model
model_ann = load_model('model_ann.h5')
new_input_layer = Input(shape=dframes.shape[1:])
new_hidden_layer1 = Dense(units=12, activation='relu',kernel_regularizer=tf.keras.regularizers.l1(0.01))(new_input_layer)
pretrained_hidden_layer2_weights = model_ann.layers[1].get_weights()
pretrained_output_layer_weights = model_ann.layers[2].get_weights()
new_hidden_layer2 = Dense(units=pretrained_hidden_layer2_weights[0].shape[1], activation='relu', weights=pretrained_hidden_layer2_weights)(new_hidden_layer1)
new_output_layer = Dense(units=pretrained_output_layer_weights[0].shape[1], activation='sigmoid', weights=pretrained_output_layer_weights)(new_hidden_layer2)
new_model = Model(inputs=new_input_layer, outputs=new_output_layer)
new_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

for layer in new_model.layers[2:]:
    layer.trainable = False
    
new_model.fit(X_train, y_train, batch_size = 20, epochs = 100)
prediction = new_model.predict(df3)
if prediction[0][0] > 0.5:
    predicted_class = "Healthy"
else:
    predicted_class = "Patient"
print("The user might fall under :", predicted_class)

y_pred = new_model.predict(X_test)
y_pred = (y_pred > 0.5)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

What is the user's age (in years): 66
Enter the number of columns you want to update (minimum 3, maximum 45): 3
Enter the name of the Feature 1: Apathy
Enter the score of the Feature 1: 2
Enter the name of the Feature 2: Clock
Enter the score of the Feature 2: 4
Enter the name of the Feature 3: UPSIT
Enter the score of the Feature 3: 22
Epoch 1/100
12/12 [==============================] - 1s 3ms/step - loss: 5.1345 - accuracy: 0.3304
Epoch 2/100
12/12 [==============================] - 0s 0s/step - loss: 1.6938 - accuracy: 0.3833
Epoch 3/100
12/12 [==============================] - 0s 592us/step - loss: 1.3205 - accuracy: 0.6388
Epoch 4/100
12/12 [==============================] - 0s 1ms/step - loss: 1.1218 - accuracy: 0.5330
Epoch 5/100
12/12 [==============================] - 0s 1ms/step - loss: 0.9813 - accuracy: 0.4626
Epoch 6/100
12/12 [==============================] - 0s 1ms/step - loss: 0.8583 - accuracy: 0.5595
Epoch 7/100
12/12 [==============================] - 0s 2ms/step -

12/12 [==============================] - 0s 1ms/step - loss: 0.5184 - accuracy: 0.8106
Epoch 80/100
12/12 [==============================] - 0s 1ms/step - loss: 0.5142 - accuracy: 0.7974
Epoch 81/100
12/12 [==============================] - 0s 1ms/step - loss: 0.5205 - accuracy: 0.7885
Epoch 82/100
12/12 [==============================] - 0s 1ms/step - loss: 0.5099 - accuracy: 0.8194
Epoch 83/100
12/12 [==============================] - 0s 1ms/step - loss: 0.5107 - accuracy: 0.8238
Epoch 84/100
12/12 [==============================] - 0s 1ms/step - loss: 0.5071 - accuracy: 0.8238
Epoch 85/100
12/12 [==============================] - 0s 1ms/step - loss: 0.5084 - accuracy: 0.8370
Epoch 86/100
12/12 [==============================] - 0s 2ms/step - loss: 0.5028 - accuracy: 0.8106
Epoch 87/100
12/12 [==============================] - 0s 3ms/step - loss: 0.5025 - accuracy: 0.8282
Epoch 88/100
12/12 [==============================] - 0s 1ms/step - loss: 0.5009 - accuracy: 0.8238
Epoch 89/100


0.8070175438596491